In [1]:
Output = c('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Processing/Output')
cur_date = "081523"

library(readxl)
library(tidyverse)
library(openxlsx)
library(vsn)

# reading in files
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, 

In [2]:
head(proteomics_25_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


# Peptide Normalization
Using Variance Stabilizing Normalization (VSN).

In [3]:
# first converting from long to wide format, which is necessary for the vsn function
wider_proteomics_25_df = proteomics_25_df %>%
# creating sample ids
    unite("SampleID", c(colnames(proteomics_25_df)[c(1,6:8)]), sep = "_") %>%
    select(-c("Gene_Name", "Description", "Intensity_pslog2", "Subject_ID")) %>%
    pivot_wider(names_from = "SampleID", values_from = "Intensity") %>%
    column_to_rownames(var = "Protein_Accession")

wider_proteomics_5_df = proteomics_5_df %>%
# creating sample ids
    unite("SampleID", c(colnames(proteomics_25_df)[c(1,6:8)]), sep = "_") %>%
    select(-c("Gene_Name", "Description", "Intensity_pslog2", "Subject_ID")) %>%
    pivot_wider(names_from = "SampleID", values_from = "Intensity") %>%
    column_to_rownames(var = "Protein_Accession")

head(wider_proteomics_25_df)

,1_C_F_25,2_C_F_25,3_C_F_25,4_C_F_25,5_C_F_25,6_C_F_25,1_C_S_25,2_C_S_25,3_C_S_25,4_C_S_25,⋯,3_P_F_25,4_P_F_25,5_P_F_25,6_P_F_25,1_P_S_25,2_P_S_25,3_P_S_25,4_P_S_25,5_P_S_25,6_P_S_25
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CO3_HUMAN,1488600000,3.8066e+09,3.9000e+09,5.3732e+09,2595200000,5780700000,4.1688e+09,2.4289e+09,5198100000,9.2868e+09,⋯,4510300000,1.0391e+10,3766600000,5460200000,2249000000,2.8799e+09,4.8758e+09,1.4966e+10,3734300000,1.6081e+10
K1C19_HUMAN,6021100000,1.9726e+10,4.1603e+09,6.4458e+09,7470700000,3719100000,6.7010e+09,1.1612e+10,6557700000,2.3940e+10,⋯,4210400000,1.3241e+10,8344100000,8590100000,6055100000,9.1549e+09,1.0989e+10,1.6041e+10,4841700000,6.4467e+09
TBB4B_HUMAN,3074000000,1.4896e+10,1.8863e+09,1.6609e+09,1519400000,1549800000,3.9525e+09,7.9426e+09,3895000000,1.5815e+10,⋯,1394300000,8.5374e+09,1532300000,2165400000,2559400000,5.2964e+09,5.9698e+09,4.1671e+09,800010000,1.7326e+09
K2C5_HUMAN,4767800000,3.5045e+09,3.4925e+09,5.3666e+09,5907300000,3072900000,5.9636e+09,4.7625e+09,6328100000,2.0851e+10,⋯,2765300000,1.1176e+10,6433300000,8647300000,6477100000,9.5103e+09,4.7240e+09,1.5121e+10,4511100000,4.0606e+09
ACTG_HUMAN,5420100000,1.5121e+10,3.4553e+09,7.1396e+09,4962400000,5649500000,6.0785e+09,1.1247e+10,5655600000,2.0502e+10,⋯,5309400000,1.4584e+10,5596300000,7149100000,3947400000,1.0762e+10,1.0041e+10,1.3861e+10,3347300000,6.1786e+09
PIGR_HUMAN,8038200000,1.9136e+10,1.0893e+10,1.4922e+10,4744600000,9297400000,1.4044e+10,1.1491e+10,7957600000,1.5351e+10,⋯,9538300000,1.7464e+10,5873800000,7215000000,7007500000,1.1390e+10,5.7456e+09,1.8687e+10,3939200000,1.4047e+10


In [4]:
normalized_25_matrix = justvsn(as.matrix(wider_proteomics_25_df))
normalized_5_matrix = justvsn(as.matrix(wider_proteomics_5_df))
head(normalized_25_matrix)

,1_C_F_25,2_C_F_25,3_C_F_25,4_C_F_25,5_C_F_25,6_C_F_25,1_C_S_25,2_C_S_25,3_C_S_25,4_C_S_25,⋯,3_P_F_25,4_P_F_25,5_P_F_25,6_P_F_25,1_P_S_25,2_P_S_25,3_P_S_25,4_P_S_25,5_P_S_25,6_P_S_25
CO3_HUMAN,30.47131,31.82586,31.86083,32.32313,31.27320,32.42860,31.95699,31.17766,32.27534,33.11253,⋯,32.07058,33.27462,31.81062,32.34631,31.06664,31.42337,32.18299,33.80097,31.79819,33.90464
K1C19_HUMAN,32.48738,34.19938,31.95404,32.58571,32.79860,31.79231,32.64173,33.43490,32.61054,34.47870,⋯,31.97131,33.62429,32.95811,33.00003,32.49550,33.09190,33.35534,33.90105,32.17287,32.58591
TBB4B_HUMAN,31.51747,33.79421,30.81291,30.62932,30.50085,30.52943,31.88012,32.88696,31.85898,33.88057,⋯,30.37689,32.99115,30.51305,31.01199,31.25316,32.30236,32.47504,31.95640,29.57544,30.69029
K2C5_HUMAN,32.15068,31.70656,31.70161,32.32136,32.45985,31.51695,32.47354,32.14907,32.55913,34.27940,⋯,31.36479,33.37968,32.58291,33.00960,32.59270,33.14684,32.13736,33.81583,32.07083,31.91905
ACTG_HUMAN,32.33567,33.81583,31.68616,32.73320,32.20839,32.39548,32.50107,33.38882,32.39703,34.25505,⋯,32.30590,33.76367,32.38183,32.73511,31.87826,33.32523,33.22518,33.69031,31.64035,32.52463
PIGR_HUMAN,32.90423,34.15557,33.34268,33.79672,32.14364,33.11418,33.70923,33.41979,32.88969,33.83761,⋯,33.15109,34.02366,32.45165,32.74835,32.70625,33.40705,32.41981,34.12132,31.87526,33.70954


There's no missing data, so I'll convert the data back to a long format. 

In [5]:
normalized_25_df = data.frame(normalized_25_matrix) %>%
    rownames_to_column(var = "Protein_Accession") %>%
    pivot_longer(cols = 2:31, names_to = "Sample_No", values_to = "Norm_Intensity") %>%
    separate(Sample_No, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration"))

normalized_5_df = data.frame(normalized_5_matrix) %>%
    rownames_to_column(var = "Protein_Accession") %>%
    pivot_longer(cols = 2:31, names_to = "Sample_No", values_to = "Norm_Intensity") %>%
    separate(Sample_No, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration"))

# dataframes automatically add X to col names if it starts with a number so removing that here
normalized_25_df$Subject_No = as.numeric(gsub("X","", as.character(normalized_25_df$Subject_No)))
normalized_5_df$Subject_No = as.numeric(gsub("X","", as.character(normalized_5_df$Subject_No)))
normalized_25_df$Concentration = as.numeric(normalized_25_df$Concentration)
normalized_5_df$Concentration = as.numeric(normalized_5_df$Concentration)

# adding back in all other data
final_normalized_25_df = full_join(normalized_25_df, proteomics_25_df[,1:8])
final_normalized_5_df = full_join(normalized_5_df, proteomics_5_df[,1:8])
final_normalized_25_df = final_normalized_25_df[,c(2,7,1,9,3:6)]
final_normalized_5_df = final_normalized_5_df[,c(2,7,1,9,3:6)]

head(final_normalized_25_df)

Joining with `by = join_by(Protein_Accession, Subject_No, Condensate,
Burn_Condition, Concentration)`
Joining with `by = join_by(Protein_Accession, Subject_No, Condensate,
Burn_Condition, Concentration)`


Subject_No,Subject_ID,Protein_Accession,Description,Condensate,Burn_Condition,Concentration,Norm_Intensity
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,F_1,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,30.47131
2,M_2,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.82586
3,M_3,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.86083
4,F_4,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,32.32313
5,F_5,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.27320
6,M_6,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,32.42860


In [6]:
# exporting
write.xlsx(final_normalized_25_df, paste0(Output,"/", "Normalized_Proteomics_Data_25ug_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(final_normalized_5_df, paste0(Output,"/", "Normalized_Proteomics_Data_5ug_", cur_date, ".xlsx"), 
           rowNames = FALSE)